# Einführung in Parallele Programmierung

Die parallele Programmierung ist ein Ansatz zur gleichzeitigen Ausführung von Aufgaben, um die Gesamtleistung und Effizienz eines Programms zu verbessern. Sie ist besonders wichtig in der heutigen Zeit, da Computer mit mehreren Prozessorkernen und -threads ausgestattet sind, die gleichzeitig arbeiten können. Parallele Programmierung zielt darauf ab, die Ressourcen eines Systems optimal zu nutzen und die Bearbeitungszeit von Aufgaben zu verkürzen.

## Warum ist parallele Programmierung wichtig?

- **Leistungssteigerung**: Durch die parallele Ausführung von Aufgaben können mehrere Berechnungen gleichzeitig stattfinden, was insgesamt zu einer schnelleren Ausführung führt.
- **Skalierbarkeit**: Mit paralleler Programmierung können Programme einfacher an unterschiedliche Hardwarekonfigurationen angepasst werden, indem mehr Prozessorkerne oder -threads genutzt werden.
- **Effizienz**: Viele Aufgaben in der heutigen Softwareentwicklung erfordern parallele Verarbeitung, um große Datenmengen effizient zu verarbeiten oder um Echtzeitreaktionen zu gewährleisten.

## Grundlegende Konzepte

- **Threads und Prozesse**: Zwei grundlegende Einheiten der parallelen Programmierung. Threads sind leichtgewichtige "leicht parallelisierbare" Einheiten, die innerhalb desselben Prozesses laufen und Ressourcen teilen. Prozesse hingegen sind unabhängige Ausführungseinheiten mit separaten Adressräumen.
- **Nebenläufigkeit vs. Parallelität**: Nebenläufigkeit beschreibt die gleichzeitige Ausführung mehrerer Aufgaben, während Parallelität die gleichzeitige Ausführung dieser Aufgaben auf mehreren Prozessorkernen oder -threads bedeutet.

**Serieller Ablauf:**
```C
+------------+        +-----------+        +------------+       
| Prozess 1  | -----> | Prozess 2 | -----> | Prozess 3  | 
+------------+        +-----------+        +------------+       
````

**Nebenläufigkeit**

![multithreading](./images/threads_multitasking_zeitscheiben.png)

(entnommen aus Python 3 von Johannes Ernesti, Peter Kaiser)

**Parallelität:**

![parallel](./images/parallel.png)

## Python und parallele Programmierung

In Python gibt es verschiedene Ansätze zur parallelen Programmierung, darunter das `concurrent.futures`-Modul, das eine einfache und effektive Möglichkeit bietet, Aufgaben parallel auszuführen. Dieses Modul abstrahiert die Details der Thread- und Prozessverwaltung und erleichtert die parallele Ausführung von Funktionen.

In diesem Notebook werden wir uns auf die Verwendung von `concurrent.futures` konzentrieren, um die Grundlagen der parallelen Programmierung in Python zu demonstrieren.


# Erstes Beispiel

Wir definieren zwei Methoden die stupide Zählen. Eine zählt aufwärts und eine abwärts.

In [ ]:
import random

def count_a():
    pass

def count_b():
    pass

Nun lassen wir sie einmal seriell zählen:

In [ ]:
count_a()
count_b()

Und jetzt einmal nebenläufig (also unecht parallel):

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.submit(count_a)
        executor.submit(count_b)

Und zum Schluss noch einmal wirklich parallel. Dazu müssen wir allerdings etwas tricksen, weil sich ipynb-Notebooks nicht wirklich hierfür eignen. In der Datei `parallel.py` sind die beiden Counter definiert und der parallele Aufruf. Wir starten diesen Aufruf in der Shell:

In [ ]:
!python ./parallel.py

Wie wir sehen ändert sich bei der nebenläufigen Programmierung kaum etwas an der Laufzeit. Aber der Counter B startet sofort nach Counter A, ohne das gewartet wird bis Counter A zu Ende ist. Bei der echten Parallelität starten nicht nur beide gleichzeitig, auch die Laufzeit ist deutlich kürzer, als beim seriellen Ablauf.

# Synchronisation und Locks

Manchmal greifen mehrere parallele Prozesse auf eine gemeinsame Ressource zu, wie etwa die Standardausgabe bei einem print()-Befehl. Dies kann zu Konflikten führen. Ein ähnliches Problem tritt im täglichen Leben auf. Stellen Sie sich vor, in einer Werkstatt teilen sich mehrere Handwerker ein einziges Werkzeug, etwa einen Bohrer. Es wäre chaotisch, wenn mehrere Handwerker gleichzeitig versuchen würden, den Bohrer zu benutzen. Stattdessen läuft es so ab: Wenn Max den Bohrer nimmt, zeigt er damit, dass er ihn gerade verwendet. Die anderen Handwerker sehen das und warten, bis Max fertig ist und das Werkzeug zurücklegt. Wenn Anna den Bohrer benötigt, wartet sie, bis Max seine Arbeit abgeschlossen hat und den Bohrer wieder verfügbar macht.

Ähnliches ist auch bei unseren Zählern passiert. Meistens zählen sie brav nacheinander. Aber hin und wieder sieht der Output so aus:
```
CountB: 95
CountA: 8
CountB: 94CountA: 9
````
Hier war dann die Standardausgabe `print()` noch von Counter B belegt und Counter A hat sich irgendwie dazwischen gemogelt. Um diese zu umgehen, kann man Ressourcen oder auch Variablen sperren /einen Lock setzen. Schauen wir uns das mal an einem Beispiel an:

In [ ]:
import concurrent.futures
import threading
import random

lock = threading.Lock()

def synchronized_count_a():
    for i in range(1, 101):
        with lock:
            print(f"CountA: {i}")
            # Verzoegerung: sinnlose Zufallszahlen erzeugen
            for _ in range(100000):
                random.randint(1, 100)

def synchronized_count_b():
    for i in range(100, 0, -1):
        with lock:
            print(f"CountB: {i}")
            # Verzoegerung: sinnlose Zufallszahlen erzeugen
            for _ in range(100000):
                random.randint(1, 100)


with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.submit(synchronized_count_a)
    executor.submit(synchronized_count_b)





Mitunter ist es nötig, dass im Code erst weitergemacht werden darf, wenn das Ergebnis von vorab parallel durchgeführten Prozessen vorhanden ist. Zum Beispiel, wenn mehrere Datenbankabfragen parallel ausgeführt werden und die Ergebnisse dieser Abfragen zusammengeführt werden müssen, bevor die nächste Berechnung erfolgen kann. In solchen Fällen ist es entscheidend, auf die Fertigstellung aller parallelen Aufgaben zu warten, um sicherzustellen, dass alle benötigten Daten vorliegen.

Ein praktisches Beispiel ist die Verarbeitung von großen Datenmengen. Stellen Sie sich vor, ein Programm muss eine große Datei in mehrere kleinere Teile aufteilen, diese Teile parallel verarbeiten und schließlich die Ergebnisse zusammenfügen. Der Ablauf könnte wie folgt aussehen:

1. Datei aufteilen: Die große Datei wird in mehrere kleinere Teile aufgeteilt.
2. Parallele Verarbeitung: Jeder Teil wird parallel verarbeitet, zum Beispiel durch das Ausführen einer bestimmten Analyse oder Transformation.
3. Ergebnisse zusammenführen: Nachdem alle Teile verarbeitet wurden, werden die Ergebnisse gesammelt und zu einem Gesamtresultat kombiniert.

Innerhalb von `concurrent.futures` gibt es zum Warten verschiende Werkzeuge:
- `Future.result()`: Blockiert, bis das Ergebnis eines einzelnen Futures verfügbar ist.
- `concurrent.futures.wait()`: Blockiert, bis alle Futures abgeschlossen sind oder eine bestimmte Bedingung erfüllt ist.
- `concurrent.futures.as_completed()`: Gibt Futures in der Reihenfolge ihrer Fertigstellung zurück und ermöglicht eine sofortige Verarbeitung der Ergebnisse.

Unten ein Beispiel mit `as_completed`.

# Bestimmung von $\pi$ durch Monte-Carlo-Simulation

Die Monte-Carlo-Methode ist eine stochastische Technik, die häufig in der numerischen Mathematik zur Lösung von Problemen verwendet wird, die deterministisch schwer zu lösen sind. In diesem Beispiel wird die Monte-Carlo-Methode verwendet, um den Wert von Pi zu approximieren. Das Grundprinzip ist, zufällige Punkte in einem Quadrat zu generieren und zu bestimmen, wie viele dieser Punkte innerhalb eines Kreises liegen, der in das Quadrat eingebettet ist.

## Vorgehensweise

1. **Generierung von zufälligen Punkten**: Es werden zufällige Punkte mit Koordinaten (x, y) im Bereich [-1, 1] generiert.
2. **Überprüfung der Punkte**: Für jeden Punkt wird überprüft, ob er innerhalb des Einheitskreises liegt. Dies wird durch die Bedingung `x^2 + y^2 <= 1` überprüft.
3. **Zählen der Punkte innerhalb des Kreises**: Die Anzahl der Punkte, die innerhalb des Kreises liegen, wird gezählt.
4. **Berechnung von Pi**: Der Anteil der Punkte, die innerhalb des Kreises liegen, im Vergleich zur Gesamtzahl der generierten Punkte, wird verwendet, um Pi zu schätzen. Da das Verhältnis der Fläche des Kreises zur Fläche des Quadrats Pi/4 beträgt, ergibt sich Pi aus diesem Anteil multipliziert mit 4.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Anzahl der Punkte, die verwendet werden sollen
num_points = 100

# Zufällige Punkte generieren (x, y) innerhalb des Quadrats [-1, 1] x [-1, 1]
points = np.random.rand(num_points, 2) * 2 - 1

# Punkte im Viertelkreis zählen (x^2 + y^2 <= 1)
inside_circle = points[np.linalg.norm(points, axis=1) <= 1]

# Pi approximieren
pi_estimate = 4 * len(inside_circle) / num_points

# Plot Setup
plt.figure(figsize=(6, 6))
plt.title(f'Approximation von Pi mittels Monte Carlo Methode\nGeschätzter Wert von Pi = {pi_estimate:.5f}')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('x')
plt.ylabel('y')

# Punkte plotten
plt.scatter(points[:, 0], points[:, 1], color='blue', s=5, label='Punkte außerhalb des Viertelkreises')
plt.scatter(inside_circle[:, 0], inside_circle[:, 1], color='red', s=5, label='Punkte innerhalb des Viertelkreises')

# Viertelkreis hinzufügen
circle = plt.Circle((0, 0), 1, color='green', fill=False, linestyle='--', linewidth=2)
plt.gca().add_patch(circle)

# Legende
plt.legend(loc='best')

# Anzeigen
plt.show()

## Parallelisierung

Um die Berechnung effizienter zu gestalten, wird die Aufgabe auf mehrere parallele Prozesse aufgeteilt:

- **Aufteilung der Stichproben**: Die Gesamtzahl der zu generierenden Stichproben wird auf mehrere parallele Aufgaben (Threads) aufgeteilt.
- **Parallele Ausführung**: Die parallelen Aufgaben werden mithilfe des `concurrent.futures.ThreadPoolExecutor` ausgeführt.
- **Sammlung und Zusammenführung der Ergebnisse**: Die Ergebnisse der parallelen Aufgaben werden gesammelt und zusammengeführt, um den endgültigen Wert von Pi zu schätzen.

Hier ist der Code, der diese Schritte umsetzt:

In [ ]:
import concurrent.futures
import random
import math

def estimate_pi(num_samples):
    inside_circle = 0
    for _ in range(num_samples):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x*x + y*y <= 1:
            inside_circle += 1
    return inside_circle

def main():
    num_samples = 10**7  # Gesamtzahl der zu generierenden Stichproben
    num_workers = 4  # Anzahl der parallelen Aufgaben

    # Teile die Stichproben auf die Anzahl der Arbeiter auf
    samples_per_worker = num_samples // num_workers

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(estimate_pi, samples_per_worker) for _ in range(num_workers)]
        
        total_inside_circle = sum(future.result() for future in concurrent.futures.as_completed(futures))

    # Schätze Pi
    pi_estimate = (4.0 * total_inside_circle) / num_samples
    print(f"Geschätzter Wert von Pi: {pi_estimate:.10f}")


main()


# Deadlocks

Um Fehler den parallelen Zugriff auf Ressourcen zu verhindern, kann man versucht sein, so viele Locks wie möglich in seine Programme zu implementieren. Allerdings wirken sich zu viele Locks negativ auf die Performance aus. Und noch dazu erhöht jeder Lock die Chance auf einen sogenannten **Dead Lock**:

- Deadlocks sind Situationen in denen ein Thread1 eine Ressource gelockt hat und nun an einem bestimmten Punkt darauf wartet, dass Thread2 eine von ihm gelockte Ressource freigibt. Thread2 hat diese Ressource gelockt und wartet seinerseits aber auf die von Thread1 gelockte Ressource. Beide warten damit bis in die Unendlichkeit. 
- Solche Deadlock-Situationen zu vermeiden ist die eigentliche Kunst der Multithreading-Programmierung.

![deadlock](./images/deadlock.png)
